In [1]:
import pandas as pd
import numpy as np
import os.path as osp


from changeit3d.external_tools.imnet.loader import initialize_and_load_imnet
from changeit3d.in_out.basics import unpickle_data
from changeit3d.utils.visualization import visualize_point_clouds_3d_v2
from changeit3d.in_out.pointcloud import pc_loader_from_npz

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
## Assumes you have download ShapeTalk and our pretrained weights (shape_latents) and you have put them under ../../data
path_to_shapetalk = '../../data/shapetalk/language/shapetalk_preprocessed_public_version_0.csv'
path_to_imnet_latents = '../../data/pretrained/shape_latents/imnet_latent_codes.pkl'
top_pc_dir = '../../data/shapetalk/point_clouds/scaled_to_align_rendering'
npc_points = 2048

In [4]:
df = pd.read_csv(path_to_shapetalk)
relevant_shapes = set(df[(df.changeit_split == "test") & df.source_object_class.isin(['chair', 'table'])]['source_uid'])
print('test shapes', len(relevant_shapes))

test shapes 1149


In [5]:
random_seed = 2022
gt_pcs = {x: pc_loader_from_npz(osp.join(top_pc_dir, x + ".npz"), 
                                n_samples=npc_points, 
                                random_seed=random_seed) for x in relevant_shapes}

In [5]:
imnet_latents = next(unpickle_data(path_to_imnet_latents))

In [6]:
# pick a random sub-sample of latents to decode.
subsample = False
if subsample:
    n_shapes = 100
    np.random.seed(2023)
    uids = np.random.choice(list(relevant_shapes), n_shapes, replace=False)
else:
    uids = relevant_shapes
    
imnet_input = {uid: imnet_latents[uid] for uid in uids}

In [ ]:
# nessary imports for imnet.eval_z to work
import mcubes, math, os, torch
from tqdm.notebook import tqdm as tqdm
imnet = initialize_and_load_imnet()

# Inference
_, imnet_pcs =\
    imnet.eval_z(imnet_input, npc_points=npc_points, optimize_mesh=False, save_output=False, skip_existing=True, output_dir='./imnet_out')

# Visualization
for uid in uids:
    display(visualize_point_clouds_3d_v2([gt_pcs[uid], imnet_pcs[uid]], fig_title=["", uid]))

In [19]:
# optionally, measure reconstruction error of SGF vs. GT
from changeit3d.evaluation.generic_metrics import chamfer_dists

all_gt = [] 
all_recon = []
for model_uid in imnet_pcs.keys():
     all_gt.append(gt_pcs[model_uid])
     all_recon.append(imnet_pcs[model_uid])     
all_gt = np.array(all_gt)
all_recon = np.array(all_recon)

print('Average Recon Error (Chamfer):', chamfer_dists(all_gt, all_recon, 128)[0])

(0.00018003920558840036,
 tensor([2.1603e-04, 9.2509e-05, 2.7472e-04, 1.2180e-04, 1.9834e-04, 2.6432e-04,
         1.5531e-04, 1.1029e-04, 2.7212e-04, 1.4641e-04, 1.6485e-04, 1.9594e-04,
         2.0757e-04, 2.0293e-04, 2.9082e-04, 1.6907e-04, 1.6391e-04, 1.9180e-04,
         2.2191e-04, 2.7460e-04, 2.7304e-04, 1.6627e-04, 1.4186e-04, 1.4821e-04,
         1.2209e-04, 3.2146e-04, 1.1894e-04, 2.4946e-04, 8.3765e-05, 1.7368e-04,
         1.5488e-04, 1.9818e-04, 1.3922e-04, 1.2126e-04, 1.3613e-04, 1.5764e-04,
         2.3393e-04, 2.9748e-04, 1.6291e-04, 1.8832e-04, 2.1748e-04, 1.9941e-04,
         1.5481e-04, 2.3709e-04, 2.2288e-04, 2.3366e-04, 2.2599e-04, 2.5736e-04,
         1.2557e-04, 1.8219e-04, 1.0965e-04, 2.3126e-04, 1.6453e-04, 1.5152e-04,
         3.0131e-04, 1.2866e-04, 2.0816e-04, 1.4232e-04, 1.3785e-04, 1.7427e-04,
         2.3276e-04, 8.1997e-05, 1.1431e-04, 1.3093e-04, 1.6422e-04, 1.9045e-04,
         1.5033e-04, 2.2911e-04, 1.4592e-04, 1.4185e-04, 7.2850e-05, 1.3033e-04,
   